In [2]:
import pandas as pd
import gurobipy as gp
import numpy as np

In [3]:
#Reading the files
roiData = pd.read_csv('ROI_data.csv')
roiData.set_index('Platform', inplace=True)

roiMatData = pd.read_csv('roi_mat.csv')

### Templates of the data:

In [4]:
roiData

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Platform,,,,,,,,,,
ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


In [5]:
roiMatData

,Unnamed: 0,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
1,February,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
2,March,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
3,April,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
4,May,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9
5,June,4.0,3.2,2.7,3.4,3.4,3.0,4.5,2.10,3.8,4.1
6,July,3.9,3.6,2.0,4.4,3.9,3.7,4.3,1.80,4.0,3.8
7,August,4.2,3.3,2.8,4.2,2.0,3.7,3.6,1.50,4.4,4.3
8,September,4.1,2.8,2.5,4.2,2.9,3.7,2.8,2.50,4.0,3.4
9,October,3.0,3.0,3.1,4.6,3.1,3.3,3.2,2.30,2.5,3.2


Assume that your company is deciding how to spend a marketing budget of $10M. You work in 
the marketing department as a data scientist and the chief marketing officer has asked you 
write a report recommending how to spread this budget among several marketing mediums. 
Your department has employed an outside consulting firm to estimate the return on investment 
(ROI) of each marketing medium under consideration. The results are in the table below, and 
also in a CSV attached to this assignment:

On top of these ROIs, your boss has decided to constrain your budget as follows:

1. The amount invested in print and TV should be no more than the amount spent on 
Facebook and Email. Surprisingly, email seems to be a great channel for reaching real 
people.
2. The total amount used in social media (Facebook, LinkedIn, Instagram, Snapchat, and 
Twitter) should be at least twice of SEO and AdWords.
3. For each platform, the amount invested should be no more than $3M.

### Q3. Formulate the marketing budget allocation problem as a linear program. Use gurobi to find the optimal budget allocation. ###

In [6]:
#initializing the variables
budget = 10000000
individualBudget = 3000000

In [7]:
# Import the Gurobi package as gp, Gurobi's Python API
import gurobipy as gp

# Initialize a Gurobi model object
q1 = gp.Model()

# Add decision variables (q1X) to the model based on the number of columns in roiData
q1X = q1.addMVar(roiData.shape[1])

# Set the objective function to maximize the sum of ROI multiplied by the respective decision variable
# The objective function aims to maximize ROI (Return On Investment)
q1.setObjective(gp.quicksum([list(roiData.loc['ROI'])[i] * q1X[i] for i in range(roiData.shape[1])]), gp.GRB.MAXIMIZE)

# Initialize a list (cons) to store constraint objects
cons = [0] * 4

# Retrieve the names of the media channels from the column headers of roiData
media = list(roiData.columns)

# Add the first constraint: The sum of all decision variables should be less than or equal to the budget
cons[0] = q1.addConstr(gp.quicksum(q1X[i] for i in range(roiData.shape[1])) <= budget)

# Add the second constraint: The sum of decision variables for 'Print' and 'TV' must be less than or equal to 
# the sum of decision variables for 'Facebook' and 'Email'
cons[1] = q1.addConstr(gp.quicksum(q1X[media.index(x)] for x in ['Print', 'TV']) <=
                       gp.quicksum(q1X[media.index(x)] for x in ['Facebook', 'Email']))

# List of social media platforms for constraint
socialMedia = ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']

# Add the third constraint: The sum of decision variables for socialMedia should be greater than or equal to 
# twice the sum of decision variables for 'SEO' and 'AdWords'
cons[2] = q1.addConstr(gp.quicksum(q1X[media.index(x)] for x in socialMedia) >=
                       2 * gp.quicksum(q1X[media.index(x)] for x in ['SEO', 'AdWords']))

# Add the fourth constraint: Each decision variable should be less than or equal to individualBudget
cons[3] = q1.addConstrs(q1X[i] <= individualBudget for i in range(roiData.shape[1]))

# Run the optimization algorithm to solve the model
q1.optimize()

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 7735U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 10 columns and 31 nonzeros
Model fingerprint: 0xfb47ec00
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+06, 1e+07]
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 10 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e+05   2.125000e+06   0.000000e+00      0s
       1    4.5600000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.560000000e+05


In [8]:
pd.DataFrame([q1.X], columns=media, index=['Budget'])

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Budget,0.0,3000000.0,0.0,1000000.0,0.0,0.0,3000000.0,0.0,0.0,3000000.0


### Q4. You are asked to compare the two optimal allocations from these two ROI estimates. 

Your boss is happy to see the promising results presented by the marketing department. 
However, your boss is also very concerned because your boss recalls being somewhat 
disappointed after following such recommendations in the past. To be cautious about the 
decision, your team has decided to get another opinion about the ROI data and rerun the 
analysis. The second consulting firm returns the estimates of the ROI data in the table below 
(also in the CSV file mentioned above). 

In [9]:
# Import the Gurobi package as gp, Gurobi's Python API
import gurobipy as gp

# Initialize a second Gurobi model object for a different optimization problem
q2 = gp.Model()

# Add decision variables (q2X) to the model based on the number of columns in roiData
q2X = q2.addMVar(roiData.shape[1])

# Set the objective function to maximize the sum of the second firm's ROI estimate multiplied by the respective decision variable
# The objective function aims to maximize this second ROI estimate
q2.setObjective(gp.quicksum([list(roiData.loc['Second Firms ROI Estimate'])[i] * q2X[i] for i in range(roiData.shape[1])]), gp.GRB.MAXIMIZE)

# Initialize a list (consQ2) to store constraint objects for this model
consQ2 = [0] * 4

# Retrieve the names of the media channels from the column headers of roiData
media = list(roiData.columns)

# Add the first constraint: The sum of all decision variables should be less than or equal to the budget
consQ2[0] = q2.addConstr(gp.quicksum(q2X[i] for i in range(roiData.shape[1])) <= budget)

# Add the second constraint: The sum of decision variables for 'Print' and 'TV' must be less than or equal to 
# the sum of decision variables for 'Facebook' and 'Email'
consQ2[1] = q2.addConstr(gp.quicksum(q2X[media.index(x)] for x in ['Print', 'TV']) <=
                         gp.quicksum(q2X[media.index(x)] for x in ['Facebook', 'Email']))

# List of social media platforms for constraint
socialMedia = ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']

# Add the third constraint: The sum of decision variables for socialMedia should be greater than or equal to 
# twice the sum of decision variables for 'SEO' and 'AdWords'
consQ2[2] = q2.addConstr(gp.quicksum(q2X[media.index(x)] for x in socialMedia) >=
                         2 * gp.quicksum(q2X[media.index(x)] for x in ['SEO', 'AdWords']))

# Add the fourth constraint: Each decision variable should be less than or equal to individualBudget
consQ2[3] = q2.addConstrs(q2X[i] <= individualBudget for i in range(roiData.shape[1]))

# Run the optimization algorithm to solve the model
q2.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 7735U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 10 columns and 31 nonzeros
Model fingerprint: 0x31bef740
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+06, 1e+07]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 10 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e+05   2.125000e+06   0.000000e+00      0s
       1    4.5600000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.560000000e+05


In [10]:
pd.DataFrame([q2.X], columns=media, index=['Budget'])

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Budget,3000000.0,0.0,0.0,1000000.0,3000000.0,3000000.0,0.0,0.0,0.0,0.0


In [11]:
pd.concat([pd.DataFrame([q1.X], columns=media, index=['First Opinion']), pd.DataFrame([q2.X], columns=media, index=['Second Opinion'])])

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
First Opinion,0.0,3000000.0,0.0,1000000.0,0.0,0.0,3000000.0,0.0,0.0,3000000.0
Second Opinion,3000000.0,0.0,0.0,1000000.0,3000000.0,3000000.0,0.0,0.0,0.0,0.0


### Q5. 
- Are the allocations the same? 
- Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)? 
- Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective? 
- Do you think the third constraint above, based on your boss’ experience, is useful?

##### <b>Q.</b> Are allocations the same?

<b> A: </b> 
No, the allocations are not the same. The platforms shifted from TV, Instagram and Email to Print, Facebook and LinkedIn while AdWords remain the constant. Additionally, the distribution between the platforms remain the same, except for the AdWords.

##### <b>Q.</b> Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)?

In [12]:
# Calculate the ROI for the second allocation using the original (first) ROI data
# This involves summing up the product of the optimized decision variables (q2.X[i]) and their corresponding ROI values
secondAllocationFirstROI = sum([q2.X[i]*roiData.loc['ROI'][i] for i in range(roiData.shape[1])]) 

# Retrieve the optimized objective value for the first allocation (which was based on the first ROI estimate)
firstAllocationFirstROI = q1.ObjVal

# Calculate the difference in ROI between the second and first allocation based on the first ROI estimate
differenceFirstROI = secondAllocationFirstROI - firstAllocationFirstROI

# Print the calculated difference in ROI between the two allocations
print("The difference in ROI between the first and second allocation is using first ROI: " + str(differenceFirstROI))

The difference in ROI between the first and second allocation is using first ROI: -204000.0


##### <b>Q.</b> Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective?

In [13]:
# Calculate the ROI for the first allocation using the second firm's ROI estimate
# This involves summing up the product of the optimized decision variables (q1.X[i]) and their corresponding second firm's ROI values
firstAllocationSecondROI = sum([q1.X[i]*roiData.loc['Second Firms ROI Estimate'][i] for i in range(roiData.shape[1])])

# Retrieve the optimized objective value for the second allocation (which was based on the second firm's ROI estimate)
SecondAllocationSecondROI = q2.ObjVal

# Calculate the difference in ROI between the first and second allocations when evaluated based on the second firm's ROI estimate
differenceSecondROI = firstAllocationSecondROI - SecondAllocationSecondROI

# Print the calculated difference in ROI between the two allocations when evaluated based on the second firm's ROI
print("The difference in ROI between the first and second allocation is using second ROI: " + str(differenceSecondROI))

The difference in ROI between the first and second allocation is using second ROI: -192000.0


##### <b>Q.</b> Do you think the third constraint above, based on your boss’ experience, is useful?

Using optimal objective function 1 assuming first ROI, assuming first ROI is correct, without third constraint. 

In [14]:
# Initialize a third Gurobi model object for another optimization problem
q31 = gp.Model()

# Add decision variables (q31X) to the model based on the number of columns in roiData
q31X = q31.addMVar(roiData.shape[1])

# Set the objective function to maximize the sum of the original ROI multiplied by the respective decision variable
# The objective function aims to maximize the ROI (Return On Investment)
q31.setObjective(gp.quicksum([list(roiData.loc['ROI'])[i] * q31X[i] for i in range(roiData.shape[1])]), gp.GRB.MAXIMIZE)

# Initialize a list (consQ31) to store constraint objects for this model
consQ31 = [0] * 3

# Retrieve the names of the media channels from the column headers of roiData
media = list(roiData.columns)

# Add the first constraint: The sum of all decision variables should be less than or equal to the budget
consQ31[0] = q31.addConstr(gp.quicksum(q31X[i] for i in range(roiData.shape[1])) <= budget)

# Add the second constraint: The sum of decision variables for 'Print' and 'TV' must be less than or equal to 
# the sum of decision variables for 'Facebook' and 'Email'
consQ31[1] = q31.addConstr(gp.quicksum(q31X[media.index(x)] for x in ['Print', 'TV']) <=
                          gp.quicksum(q31X[media.index(x)] for x in ['Facebook', 'Email']))

# List of social media platforms for constraint
socialMedia = ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']

# Add the third constraint: The sum of decision variables for socialMedia should be greater than or equal to 
# twice the sum of decision variables for 'SEO' and 'AdWords'
consQ31[2] = q31.addConstr(gp.quicksum(q31X[media.index(x)] for x in socialMedia) >=
                           2 * gp.quicksum(q31X[media.index(x)] for x in ['SEO', 'AdWords']))

# Run the optimization algorithm to solve the model
q31.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 7735U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 10 columns and 21 nonzeros
Model fingerprint: 0x57c73cc6
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+07, 1e+07]
Presolve removed 1 rows and 7 columns
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3900000e+29   3.000000e+30   2.780000e-01      0s
       2    4.6500000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.650000000e+05


Using optimal objective function 1 assuming first ROI, assuming first ROI is correct, without third constraint. 

In [15]:
# Initialize a fourth Gurobi model object for another optimization problem
q32 = gp.Model()

# Add decision variables (q32X) to the model based on the number of columns in roiData
q32X = q32.addMVar(roiData.shape[1])

# Set the objective function to maximize the sum of the second firm's ROI estimate multiplied by the respective decision variable
# The objective function aims to maximize the ROI (Return On Investment) based on the second firm's estimates
q32.setObjective(gp.quicksum([list(roiData.loc['Second Firms ROI Estimate'])[i] * q32X[i] for i in range(roiData.shape[1])]), gp.GRB.MAXIMIZE)

# Initialize a list (consQ32) to store constraint objects for this model
consQ32 = [0] * 3

# Retrieve the names of the media channels from the column headers of roiData
media = list(roiData.columns)

# Add the first constraint: The sum of all decision variables should be less than or equal to the budget
consQ32[0] = q32.addConstr(gp.quicksum(q32X[i] for i in range(roiData.shape[1])) <= budget)

# Add the second constraint: The sum of decision variables for 'Print' and 'TV' must be less than or equal to 
# the sum of decision variables for 'Facebook' and 'Email'
consQ32[1] = q32.addConstr(gp.quicksum(q32X[media.index(x)] for x in ['Print', 'TV']) <=
                          gp.quicksum(q32X[media.index(x)] for x in ['Facebook', 'Email']))

# List of social media platforms for constraint
socialMedia = ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']

# Add the third constraint: The sum of decision variables for socialMedia should be greater than or equal to 
# twice the sum of decision variables for 'SEO' and 'AdWords'
consQ32[2] = q32.addConstr(gp.quicksum(q32X[media.index(x)] for x in socialMedia) >=
                           2 * gp.quicksum(q32X[media.index(x)] for x in ['SEO', 'AdWords']))

# Run the optimization algorithm to solve the model
q32.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 7735U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 10 columns and 21 nonzeros
Model fingerprint: 0x2ba83467
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+07, 1e+07]
Presolve removed 1 rows and 7 columns
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3900000e+29   3.000000e+30   2.780000e-01      0s
       2    4.6500000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.650000000e+05


In [16]:
roiData

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Platform,,,,,,,,,,
ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


In [17]:
pd.concat([pd.DataFrame([q1.X], columns=media, index=['First Opinion']), 
           pd.DataFrame([q2.X], columns=media, index=['Second Opinion']),
           pd.DataFrame([q31.X], columns=media, index=['Third Constraint Check 1']),
           pd.DataFrame([q32.X], columns=media, index=['Third Constraint Check 2'])])

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
First Opinion,0.0,3000000.0,0.0,1000000.0,0.0,0.0,3000000.0,0.0,0.0,3000000.0
Second Opinion,3000000.0,0.0,0.0,1000000.0,3000000.0,3000000.0,0.0,0.0,0.0,0.0
Third Constraint Check 1,0.0,5000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0
Third Constraint Check 2,5000000.0,0.0,0.0,0.0,5000000.0,0.0,0.0,0.0,0.0,0.0


<b>Observation:</b>
- Looking at the allocation in the scenario where constraint 3 doesn't exist, we see that all of the allocation goes to the platforms with maximum return.

But how does it fare with the case with budget restraint on each platform? To quantify it, let us look at the difference between the returns obtained in the case without constraint and with constraint.

In [18]:
q31.ObjVal - q1.ObjVal

9000.0

In [19]:
q32.ObjVal - q2.ObjVal

9000.0

Upon analysis, we identify a variation in returns amounting to USD 9,000.

The significance of this discrepancy hinges on the company's valuation of a USD 9,000 shortfall. It's crucial to remember that these returns are based on estimated figures, not actualized gains. Therefore, the final decision remains subjective, influenced by managerial discretion. As observed in the first two rows of the provided table, the manager's choice ensures promotional diversity across platforms, potentially at the expense of an approximate USD 9,000. Given this context, the manager's decision, reinforced by their experience, appears to be judicious.

### Q6. To explore this further perform some analysis of how your optimal allocation would change based on changes in the ROI data. Use the first ROI data as your starting point. By how much could each advertising medium’s ROI increase or decrease and still result in the same optimal allocation you found in step (3)?

In [20]:
# Initialize empty lists to hold the advertising medium name, low, current and high objective values, and the allocation values
advMedium = []
objLowList = []
objCurrentList = []
objHighList = []
allocationList = []

# Loop through each variable in the Gurobi model q1
for i in range(len(q1.getVars())):
    # Get the name of the advertising medium from roiData
    varName = roiData.columns[i]
    
    # Calculate the low sensitivity range for the objective, ensuring it is at least zero, and convert to percentage
    objLow = q1.getVars()[i].SAObjLow * 100
    
    # Calculate the current objective value and convert to percentage
    objCurrent = q1.getVars()[i].Obj * 100
    
    # Calculate the high sensitivity range for the objective, ensuring it is at most one, and convert to percentage
    objHigh = q1.getVars()[i].SAObjUp * 100
    
    # Create a DataFrame to hold the allocation values for 'First Opinion', which are the values of decision variables
    allocationDf = pd.DataFrame([q1.X], columns=media, index=['First Opinion'])
    
    # Extract the allocation amount for this specific advertising medium
    allocation = allocationDf[varName][0]
    
    # Append the collected information to the respective lists
    advMedium.append(varName)
    objLowList.append(f"{objLow:.1f}%")
    objCurrentList.append(f"{objCurrent:.1f}%")
    objHighList.append(f"{objHigh:.1f}%")
    allocationList.append(f"${allocation:,.0f}")

# Create a DataFrame to hold all the sensitivity analysis data
sensitivityDf = pd.DataFrame({
    'Advertising Medium': advMedium,
    'Obj Low': objLowList,
    'Obj Current': objCurrentList,
    'Obj High': objHighList,
    'Allocation': allocationList
})

# Show the sensitivity DataFrame
sensitivityDf

,Advertising Medium,Obj Low,Obj Current,Obj High,Allocation
0,Print,-inf%,3.1%,4.9%,$0
1,TV,3.9%,4.9%,6.2%,"$3,000,000"
2,SEO,-inf%,2.4%,3.9%,$0
3,AdWords,3.3%,3.9%,4.6%,"$1,000,000"
4,Facebook,-inf%,1.6%,2.9%,$0
5,LinkedIn,-inf%,2.4%,3.9%,$0
6,Instagram,3.9%,4.6%,inf%,"$3,000,000"
7,Snapchat,-inf%,2.6%,3.9%,$0
8,Twitter,-inf%,3.3%,3.9%,$0
9,Email,2.9%,4.4%,inf%,"$3,000,000"


### Q7. Your boss has gained permission to reinvest half of the return. For example, if the marketing obtains a 4% return in January, the budget of February will be USD 10M + USD 10M × 4% × 50% = USD 10.2M. The monthly ROI for next year is given in an attached csv file. The three constraints given by your boss are still in place for each month. What is the optimal allocation for each month?

In [21]:
#Rename index column name to 'Month'
roiMatData.rename(columns={'Unnamed: 0': 'Month'}, inplace=True)
roiMatData.set_index('Month', inplace=True)

In [22]:
roiMatData

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Month,,,,,,,,,,
January,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
February,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
March,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
April,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
May,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9
June,4.0,3.2,2.7,3.4,3.4,3.0,4.5,2.10,3.8,4.1
July,3.9,3.6,2.0,4.4,3.9,3.7,4.3,1.80,4.0,3.8
August,4.2,3.3,2.8,4.2,2.0,3.7,3.6,1.50,4.4,4.3
September,4.1,2.8,2.5,4.2,2.9,3.7,2.8,2.50,4.0,3.4


In [23]:
# Set an initial budget amount
initialBudget = 10000000

# Create a list of months based on the index of roiMatData DataFrame
months = list(roiMatData.index)

# Initialize empty lists to store monthly allocation details and ROI
monthsAllocation = []
monthsROI = []

# Initialize variable for storing ROI of each month
monthlyROI = 0

# Initialize list for storing monthly allocation
monthlyAllocation = []

# Loop through each month
for month in months:
    # Print the current month for tracking progress
    print("Month: " + month)
    
    # Append a dictionary with month, initial budget, and monthly ROI to monthsAllocation list
    monthsAllocation.append({'Month': month, 'Budget': initialBudget, 'ROI' : monthlyROI})
    
    # Create a Gurobi model for optimization
    q5 = gp.Model()
    
    # Add decision variables for the Gurobi model based on the number of columns in roiMatData for that month
    q5X = q5.addMVar(roiMatData.loc[month].shape[0])
    
    # Set the objective function to maximize the sum of ROI multiplied by decision variables
    q5.setObjective(gp.quicksum([list(roiMatData.loc[month])[i]/100 * q5X[i] for i in range(roiMatData.loc[month].shape[0])]), gp.GRB.MAXIMIZE)
    
    # Initialize a list to hold constraint objects
    consQ5 = [0] * 4
    
    # Extract column names to represent different media channels
    media = list(roiMatData.columns)
    
    # Add constraints for the Gurobi model
    consQ5[0] = q5.addConstr(gp.quicksum(q5X[i] for i in range(roiMatData.loc[month].shape[0])) <= initialBudget)
    consQ5[1] = q5.addConstr(gp.quicksum(q5X[media.index(x)] for x in ['Print', 'TV']) <=
                            gp.quicksum(q5X[media.index(x)] for x in ['Facebook', 'Email']))
    
    # Define the list of social media platforms
    socialMedia = ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']
    
    # Add another constraint for social media allocation
    consQ5[2] = q5.addConstr(gp.quicksum(q5X[media.index(x)] for x in socialMedia) >=
                                2* gp.quicksum(q5X[media.index(x)] for x in ['SEO', 'AdWords']))
    
    # Add constraints for individual budgets for each channel
    consQ5[3] = q5.addConstrs(q5X[i] <= individualBudget for i in range(roiMatData.loc[month].shape[0]))
    
    # Run the optimization
    q5.optimize()
    
    # Store the optimized objective value as the monthly ROI
    monthlyROI = q5.ObjVal
    
    # Append monthly allocation details
    monthlyAllocation.append({'Month': month, 'Allocation': list(q5.X) + [sum(list(q5.X))]})
    
    # Reset the initial budget to $10,000,000
    initialBudget = 10000000
    
    # Update the budget based on the obtained monthly ROI, only if ROI is positive
    if monthlyROI >= 0:
        initialBudget = initialBudget + 0.5*monthlyROI
    else:
        # Print a message if the monthly ROI is negative
        print("Monthly ROI for " + month + " is negative.")

Month: January
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 7735U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 10 columns and 31 nonzeros
Model fingerprint: 0x07269b35
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 4e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+06, 1e+07]
Presolve removed 10 rows and 1 columns
Presolve time: 0.01s
Presolved: 3 rows, 9 columns, 19 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+05   2.125000e+06   0.000000e+00      0s
       2    3.7300000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.730000000e+05
Month: February
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 7735U with Radeon Graphics, ins

In [24]:
# Initialize an empty list to store formatted allocation data
formattedData = []

# Loop through each entry in the 'monthlyAllocation' list
for entry in monthlyAllocation:
    # Format each allocation amount in the entry to display in millions and append it to 'formattedRow'
    formattedRow = ["${:,.2f} M".format(value/1e6) if value != 0 else '-' for value in entry['Allocation']]
    
    # Append the formatted row to 'formattedData'
    formattedData.append(formattedRow)

# Create a list of column names by extending the existing 'roiMatData' columns with 'Monthly Allocation'
roiMatData_columns = list(roiMatData.columns) + ['Monthly Allocation']

# Create a new DataFrame 'dfAllocation' using the 'formattedData', with column names and row index specified
dfAllocation = pd.DataFrame(formattedData, columns=roiMatData_columns, index=[entry['Month'] for entry in monthlyAllocation])

# Reset the DataFrame index and move the existing index to a new column named 'index'
dfAllocation.reset_index(inplace=True)

# Rename the column 'index' to 'Month'
dfAllocation.rename(columns={'index': 'Month'}, inplace=True)

# Set 'Month' as the DataFrame index
dfAllocation.set_index('Month', inplace=True)

# Display the DataFrame 'dfAllocation'
dfAllocation

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Monthly Allocation
Month,,,,,,,,,,,
January,$3.00 M,-,-,$1.33 M,-,-,$2.67 M,-,-,$3.00 M,$10.00 M
February,$3.00 M,-,-,$2.40 M,$3.00 M,-,-,-,$1.79 M,-,$10.19 M
March,-,-,-,$3.00 M,-,$3.00 M,$1.20 M,-,$3.00 M,-,$10.20 M
April,-,-,-,$3.00 M,-,$3.00 M,$3.00 M,-,$1.20 M,-,$10.20 M
May,$1.20 M,-,-,-,-,-,$3.00 M,-,$3.00 M,$3.00 M,$10.20 M
June,$3.00 M,-,-,-,-,-,$3.00 M,-,$1.21 M,$3.00 M,$10.21 M
July,-,-,-,$3.00 M,$1.21 M,-,$3.00 M,-,$3.00 M,-,$10.21 M
August,$2.71 M,-,-,$1.50 M,-,-,-,-,$3.00 M,$3.00 M,$10.21 M
September,$0.61 M,-,-,$3.00 M,-,$3.00 M,-,-,$3.00 M,$0.61 M,$10.22 M


### Q8. A stable budget is defined as a monthly allocation such that for each platform the monthly change in spend is no more than $1M. Is the allocation you found stable? If it isn’t, you do not need to solve a new optimization model. Describe how my might model this?

No, the allocation we found was not stable.

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

# Convert formatted strings to float
def convert_to_float(cell_value):
    if cell_value == '-':
        return 0.0
    else:
        return float(cell_value.replace("$", "").replace(" M", ""))

dfAllocation_numeric = dfAllocation.applymap(convert_to_float)

def highlight_increase(data):
    style = pd.DataFrame('', index=data.index, columns=data.columns)
    for col in data.columns:
        for row in range(1, len(data)):
            if abs(data[col].iloc[row] - data[col].iloc[row - 1]) >= 1.0:
                style.loc[style.index[row], col] = 'background-color: red'
    return style

# Apply the styling
styled_df = dfAllocation_numeric.style.apply(highlight_increase, axis=None)
styled_df

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Monthly Allocation
Month,,,,,,,,,,,
January,3.000000,0.000000,0.000000,1.330000,0.000000,0.000000,2.670000,0.000000,0.000000,3.000000,10.000000
February,3.000000,0.000000,0.000000,2.400000,3.000000,0.000000,0.000000,0.000000,1.790000,0.000000,10.190000
March,0.000000,0.000000,0.000000,3.000000,0.000000,3.000000,1.200000,0.000000,3.000000,0.000000,10.200000
April,0.000000,0.000000,0.000000,3.000000,0.000000,3.000000,3.000000,0.000000,1.200000,0.000000,10.200000
May,1.200000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,3.000000,3.000000,10.200000
June,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1.210000,3.000000,10.210000
July,0.000000,0.000000,0.000000,3.000000,1.210000,0.000000,3.000000,0.000000,3.000000,0.000000,10.210000
August,2.710000,0.000000,0.000000,1.500000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,10.210000
September,0.610000,0.000000,0.000000,3.000000,0.000000,3.000000,0.000000,0.000000,3.000000,0.610000,10.220000


- Adding a new constraint of mod(allocation(older model) - allocation(newer model))<=1000000
- All the other constraints and modelling would remain the same.